<a href="https://colab.research.google.com/github/KamonohashiPerry/TensorFlow/blob/master/TensorFlowTutorial_text_classification_movielens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# TensorFlowのバージョンが古いのでアンイストール
!pip uninstall tensorflow

Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0


In [5]:
# TensorFlowのバージョンの新しいバージョンをインストール（終わったらランタイムの再起動を行う）
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 39kB/s 
     |████████████████████████████████| 450kB 55.3MB/s 
     |████████████████████████████████| 3.8MB 54.6MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf

In [0]:
!pip install -q tensorflow-hub

In [0]:
!pip install -q tensorflow-datasets

In [2]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('Version: ', tf.__version__)
print('Eager mode: ', tf.executing_eagerly())
print('Hub version: ', hub.__version__)
print('GPU is', 'available' if tf.config.experimental.list_physical_devices('GPU') else 'NOT AVAILABLE')

Version:  2.1.0
Eager mode:  True
Hub version:  0.7.0
GPU is available


In [0]:
# データセットを6:4で訓練用とテスト用に分割
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name='imdb_reviews',
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True
)

In [4]:
# データの探索
type(train_data)

tensorflow.python.data.ops.dataset_ops._OptionsDataset

In [15]:
# 10件ほどデータを抽出する
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by

In [34]:
type(train_examples_batch[0])

tensorflow.python.framework.ops.EagerTensor

In [37]:
# 値を抽出
print(train_examples_batch[0].numpy())
print(train_labels_batch[0].numpy())

b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by a journalist rather than a novelist. It's not really

In [48]:
# モデルの構築
# 学習済みの分散表現を用いる
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [51]:
model = tf.keras.Sequential()
model.add(hub_layer) # 学習済みモデルを使う層
model.add(tf.keras.layers.Dense(16, activation='relu')) # 16個からなる隠れ層
model.add(tf.keras.layers.Dense(1, activation='sigmoid')) # 一つの出力結果となるように変換する層

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
# 損失関数と最適化手法
# 損失関数：binary_crossentropy（mean_squared_errorなどもあるが）
# 最適化手法：Adam, Adamax, Nadamなどがある。（https://keras.io/optimizers/）
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [55]:
# モデルの訓練
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 5s 179ms/step - loss: 0.7078 - accuracy: 0.5727 - val_loss: 0.6571 - val_accuracy: 0.6195
Epoch 2/20
30/30 [==============================] - 5s 154ms/step - loss: 0.6365 - accuracy: 0.6479 - val_loss: 0.6178 - val_accuracy: 0.6631
Epoch 3/20
30/30 [==============================] - 5s 157ms/step - loss: 0.6039 - accuracy: 0.6803 - val_loss: 0.5910 - val_accuracy: 0.6887
Epoch 4/20
30/30 [==============================] - 5s 165ms/step - loss: 0.5729 - accuracy: 0.7101 - val_loss: 0.5639 - val_accuracy: 0.7156
Epoch 5/20
30/30 [==============================] - 5s 159ms/step - loss: 0.5441 - accuracy: 0.7383 - val_loss: 0.5364 - val_accuracy: 0.7377
Epoch 6/20
30/30 [==============================] - 5s 164ms/step - loss: 0.5119 - accuracy: 0.7618 - val_loss: 0.5085 - val_accuracy: 0.7603
Epoch 7/20
30/30 [==============================] - 5s 170ms/step - loss: 0.4816 - accuracy: 0.7845 - val_loss: 0.4789 - val_accuracy: 0.7788
Epoch 

In [56]:
# モデルの評価
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.316
accuracy: 0.865
